# Estimate InSAR correlated noise covariance

Gareth Funning, University of California, Riverside

Noise in InSAR data is not 'white' $-$ which is to say, the noise in each pixel is not independent of the others. Rather, it is spatially correlated. Given what causes it, predominantly troposphere water vapor, this is not surprising. Water vapor bodies can be several kilometers to tens of kilometers across, and thus affect interferometric phase over similar length scales.

While we do not currently have the means to remove all of the tropospheric water vapor signal from our interferograms, we can at least quantify the effect it may have on our data. Specifically, we can measure the amplitude of any water vapor signal and the length scale it correlates over in our interferograms. If we can estimate an average covariance-vs-distance relationship, we can use this to weight our data appropriately, or to simulate realistic correlated noise. The basic idea, from Ramon Hanssen in his 2001 book, is that one can estimate the covariance change with distance by first estimating the autocorrelation of an area of the interferogram (i.e. correlating it with itself, in two dimensions), and then radially averaging it. Then, we can fit a function, usually some kind of exponential function, to that average, and use that to calculate covariances between any pair of points.

This specific implementation is based on [Tim Wright's 2003 paper on the Nenana Mountain, Alaska earthquake](https://doi.org/10.1029/2003GL018014), also used (and explained better, I think) in [my 2005 paper on the Bam, Iran earthquake](https://doi.org/10.1029/2004JB003338).

## 0. Dependencies

You need these things! Mostly mathy things.

In [ ]:
# we need these things
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from scipy.fft import fft2, ifft2
from scipy.fftpack import fftshift
from scipy.optimize import minimize

## 1. Inputs

Which files do you want to apply this analysis to? We need an interferogram and a way (or ways) to mask out bad pixels. And paths to those files.

In [ ]:
# files
ifgpath="/home/gareth/work/quakes/elazig/merged/"                     # directory with your interferogram
ifgname="filt_topophase.unw.geo"                                      # interferogram file
corname="topophase.cor.geo"                                           # correlation file
mskname="water.msk"                                                   # water mask file (optional)
use_water_mask=True

# parameters
wavel = 0.0555     # radar wavelength in m
cor_thresh = 0.25  # correlation threshold between 0 (bad) and 1 (perfect)
pixsize=30         # interferogram pixel size in m

## 2. Load in the data

We can use gdal for this. We will also convert phase to displacement.

In [ ]:
# load in your data manually!

# unwrapped interferogram
ifgfile=gdal.Open(ifgpath+ifgname,gdal.GA_ReadOnly)   # open with gdal
rb = ifgfile.GetRasterBand(2)                         # phase info is in band 2
ifg = rb.ReadAsArray()                                # read it in as a number array

# convert to meters
ifg*=wavel/4/np.pi                # change values of sc.displacement in place

# let's plot it to have a look
### matplotlib inline
fig, ax = plt.subplots(figsize=(10,10))        # new figure called "ax1"
im = ax.imshow(ifg,origin='upper')             # plot displacements with origin at upper left (ISCE default)
fig.colorbar(im)                               # plot a color bar!
plt.show()

## 3. Mask and crop the data

Apply your water mask (if you have one), and mask out low correlation pixels. Then apply a crop (of your choice) $-$ make sure to choose an area that does not have your primary signal of interest!

In [ ]:
# specify crop bounds (in image coordinates, e.g. from mdx, in pixels from the top left)
# for the full scene, set minima to 0 and maxima to -1
ymin=0
ymax=3000
xmin=0
xmax=-1

# correlation file
corfile=gdal.Open(ifgpath+corname,gdal.GA_ReadOnly)   # open with gdal
rb = corfile.GetRasterBand(2)                         # correlation info is in band 2
cor = rb.ReadAsArray()                                # read it in as a number array
       
# apply the water mask, if you have one
if use_water_mask==True:
    # water mask file
    mskfile=gdal.Open(ifgpath+mskname,gdal.GA_ReadOnly)  
    rb = mskfile.GetRasterBand(1)                         # only one band here
    msk = rb.ReadAsArray()
    ifg[msk<1]=np.nan                                     # mask out water

# convert displacements to meters and mask low correlation pixels
ifg[cor<cor_thresh]=np.nan          # mask out low correlation

# get metadata for the image: frame size, pixel size and reference point
gt = corfile.GetGeoTransform()    # geotransform information
nrows=corfile.RasterYSize

# sort out the maximum crop size in the y direction
if ymax==-1:
    ymax=nrows
    
# crop the ifg
cropifg=ifg[ymin:ymax,xmin:xmax]

# and plot it
fig, ax = plt.subplots(figsize=(10,6))        # new figure called "ax1"
im = ax.imshow(cropifg,origin='upper')             # plot displacements with origin at upper left (ISCE default)
fig.colorbar(im)                               # plot a color bar!
plt.show()

## 4. Flatten your cropped interferogram

Any ramp in the cropped data will be the largest correlated signal. We don't want to include that in our analysis! So we will flatten the cropped data by subtracting out a best-fitting plane.

In [ ]:
# flatten the interferogram crop

# make meshgrids for x and y pixel numbers
nrows,ncols = np.shape(cropifg)
xx,yy=np.meshgrid(np.arange(0,ncols,1),np.arange(0,nrows,1))
# set nulls to zero, make sparse and reshape to column
xx = xx.astype('float')
yy = yy.astype('float')

# nan out the nan pixels in your grids
xx[np.isnan(cropifg)]=np.nan
yy[np.isnan(cropifg)]=np.nan

# make vectors of pixel numbers and cropped ifg numbers
xx1d = xx.ravel()
xx1d = xx1d[~np.isnan(xx1d)]
yy1d = yy.ravel()
yy1d = yy1d[~np.isnan(yy1d)]
cropifg1d = cropifg.ravel()
cropifg1d = cropifg1d[~np.isnan(cropifg1d)]

# invert for a best-fitting plane
A = np.vstack((np.ones_like(xx1d),xx1d,yy1d)).T
ATA=np.matmul(A.T,A)
ATd=np.matmul(A.T,cropifg1d.T)
planeparams=np.matmul(np.linalg.inv(ATA),ATd.T)

# make your best-fitting plane and subtract it
plane = planeparams[0]+xx*planeparams[1]+yy*planeparams[2] 
flatcropifg = cropifg-plane

# and plot it
fig, ax = plt.subplots(figsize=(10,6))         # new figure called "ax1"
im = ax.imshow(flatcropifg,origin='upper')     # plot displacements with origin at upper left (ISCE default)
fig.colorbar(im)                               # plot a color bar!
plt.show()

## 5. Calculate the autocorrelation of your crop

The crux of the analysis is that we want to autocorrelate our data. We can do this either in the Fourier domain (fast) or in the spatial domain (slow). I strongly recommend (and have coded up) the Fourier approach $-$ which involves forward and inverse 2d Fourier transforms, plus some messing around with shifting the grid. After that, we will calculate some distances from the center of the autocorrelation grid to all of the other points on the grid.

In [ ]:
# autocorrelation in the Fourier domain

# the fft does not like nans
flatcropifgzeros=flatcropifg
flatcropifgzeros[np.isnan(flatcropifg)]=0

# take the 2d fft, extract the phase spectrum, inverse 2d fft, shift, normalize
fft_flatgrid = fft2(flatcropifgzeros)
pspec = np.real(fft_flatgrid)**2 + np.imag(fft_flatgrid)**2
autocorr_grid = ifft2(pspec)
autocorr_grid = fftshift(np.real(autocorr_grid))/np.count_nonzero(flatcropifgzeros)

xcent = np.floor(ncols/2)+1 # spectrally, the grid is not doubled in size
ycent = np.floor(nrows/2)+1 

# plot the autocorrelation grid
fig, ax = plt.subplots(figsize=(10,6))                 # new figure called "ax1"
im = ax.imshow(np.real(autocorr_grid),origin='upper')  # plot grid with origin at upper left (ISCE default)
fig.colorbar(im)                                       # plot a color bar!
plt.show()

# make grid of distances from centre 
xx, yy = np.meshgrid(np.arange(0,ncols,1),np.arange(0,nrows,1))
radial_dist = np.sqrt((xx-xcent)**2+(yy-ycent)**2)*pixsize;
rd = np.ravel(radial_dist)
acg = np.ravel(autocorr_grid)

# cut the autocorrelation grid and distance vector in half
rdlen=nrows+np.ceil(len(rd)/2) # only need 1st half of image (symmetry)
rdlen=int(rdlen)

rd = rd[0:rdlen] # only need 1st half of image (symmetry)
acg= acg[0:rdlen]

## 6. Calculate the radial average of the autocorrelation and fit a function to it

We are close to having a covariance vs distance relationship! We will calculate the radial average of the autocorrelation function, to get covariance as a function of distance, and then try a couple of different functions to fit to it. Exponential functions tend to fit okay, usually modifying them with a cosine function works best.

In [ ]:
# what is our maximum autocorrelation?
maxacg=np.max(acg)
maxr = np.ceil(np.max(rd))
print("maximum autocorrelation = {0:g} m^2".format(maxacg))
print("   corresponding to a standard deviation of = {0:f} m".format(np.sqrt(maxacg)))

print(' ')

# calculate average cov vs dist profile

# let's bin by twice the pixel size
w = pixsize*2    # bin width

# we should only fit a function to half of the autocorrelation grid (the shorter half)
if (xcent<ycent):
    maxdist = xcent*pixsize    
else:
    maxdist = ycent*pixsize
    
# truncate the radial distances and autocorrelations to this maximum distance
rdtrunc=rd[rd<maxdist]
acgtrunc=acg[rd<maxdist]

# bin all the values by distance
rdbin=np.int32(np.ceil(rdtrunc/w))    # classify values of rd according to bin number
cvdav = np.zeros((max(rdbin),2))
maxbin = np.max(rdbin)-1;

# do the averaging, bin by bin
for b in np.arange(0,maxbin+1,1):
    cvdav[b,1] = np.mean(acgtrunc[np.where(rdbin==b+1)])
    cvdav[b,0] = b*w

# now, let's fit something to that"

# define a penalty function for an exponential fit
def pendiffexp(params, cvdav):
    a = cvdav[0,1]
    b = params[0]
    eff = cvdav[:,1]-a*np.exp(-b*cvdav[:,0])
    f=np.linalg.norm(eff,2)
    return f

# define a penalty function for an exponential + cosine fit
def pendiffexpcos(params, cvdav):
    a = cvdav[0,1]
    b = params[0]
    c = params[1]
    eff = cvdav[:,1]-a*np.exp(-b*cvdav[:,0])*np.cos(c*cvdav[:,0])
    f=np.linalg.norm(eff,2)
    return f

# guess a trial solution
exp_trial=np.array((0.00015))             # try something in the realm of 10-4 m^-1
expcos_trial=np.array((0.0002, 0.0002))  # try something in the realm of 10-4 m^-1

# use the minimize function to fit something (feel free to change the method)
exp_solution=minimize(pendiffexp,exp_trial,cvdav,method = 'Nelder-Mead')
a=cvdav[0,1]
b=exp_solution.x[0]
exp_graph=a*np.exp(-b*rd)
print("exponential function, alpha = {0:g} m^-1".format(b))
print("   1/alpha = {0:f} m".format(1/b))
print(' ')


expcos_solution=minimize(pendiffexpcos,expcos_trial,cvdav)
a=cvdav[0,1]
b=expcos_solution.x[0]
c=expcos_solution.x[1]
expcos_graph=a*np.exp(-b*rd)*np.cos(c*rd)
print("exponential cosine function, alpha = {0:g} m^-1, beta = {1:g} m^-1".format(b,c))
print("   1/alpha = {0:f} m".format(1/b))


## 7. Plot the results!

With all that done, we can plot the results $-$ the range of autocorrelation values, the radial average, the different functional fits. We can also plot the e-folding value $-$ (1/e)\* the maximum covariance $-$ and the corresponding e-folding wavelength (the distance over which the covariance drops to the e-folding value). See which function seems to represent the average covariance-distance relationship the best!

In [ ]:
# plot the outcome!
plt.scatter(rd,acg,c='lightgray')
plt.plot(cvdav[:,0], cvdav[:,1],c='black',label='average')  # Plot the chart
plt.plot(rd, exp_graph,c='red',label='exp')  # Plot the chart
plt.plot(rd, expcos_graph,c='blue',label='expcos')  # Plot the chart

# mark on the e-folding value
efold=maxacg/np.exp(1)
plt.plot([0,maxr],[efold,efold],c='dimgray',ls='--')

# mark on the e-folding wavelength
idx = (np.abs(cvdav[:,1]-efold)).argmin()
plt.plot([cvdav[idx,0],cvdav[idx,0]],[np.min(acg),maxacg],c='dimgray',ls='--')
print('e-folding value: {0:g} m^2, e-folding distance: {1:f} m'.format(efold,cvdav[idx,0]))

plt.xlabel("distance (m)")  
plt.ylabel("covariance (m^2)")  
plt.title("max covariance: {0:g} m^2".format(maxacg))
plt.legend()
plt.show()  # display

Remember those numbers as we will need them later on...

## 8. References

Hanssen, R. F., 2001, Radar interferometry: Data interpretation and error analysis, Kluwer, Dordrecht, 308 pp.

Wright, T. J., Z. Lu and C. Wicks, 2003, Source model for the $M_w$ 6.7, 23 October 2002, Nenana Mountain
Earthquake (Alaska) from InSAR, Geophys. Res. Lett., 30, 1974, https://doi.org/10.1029/2003GL018014

Funning, G. J., B. Parsons, T. J. Wright, J. A. Jackson and E. J. Fielding, 2005, Surface displacements and source parameters of the 2003 Bam, Iran earthquake from Envisat Advanced Synthetic Aperture Radar imagery, J. Geophys. Res. Solid Earth, 110 (B9), B09406, https://doi.org/10.1029/2004JB003338